In [49]:
# importing pandas
import pandas as pd
# load data from dataset
dataset = pd.read_csv("C:\\Users\\Admin\\Documents\\data_mining\\Market_Basket_Optimisation.csv")
# size of dataset
dataset.shape

(1203, 20)

In [50]:
dataset.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# importing numpy
import numpy as np
# Tạo 1 mảng lưu tất cả các dữ liệu trong dataset
transaction = []
for i in range(0, dataset.shape[0]):
    for j in range(0, dataset.shape[1]):
        transaction.append(dataset.values[i,j])
# chuyển thành mảng numpy
transaction = np.array(transaction)
print(transaction)

['burgers' 'meatballs' 'eggs' ... 'nan' 'nan' 'nan']


In [52]:
#  Chuyển đổi mảng numpy thành dataframe
df = pd.DataFrame(transaction, columns=["items"]) 
# Tạo biến đếm, đếm tần suất xuất hiện của các item
df["incident_count"] = 1 
#  Xóa các item có kiểu dữ liệu null 
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)
# tạo df đếm số lượng xuất hiện của các item
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
#  lấy top 5 item xuất hiện nhiều nhất
df_table.head(5).style.background_gradient(cmap='Blues')

,items,incident_count
0,mineral water,292
1,eggs,236
2,chocolate,228
3,spaghetti,218
4,french fries,197


Vẽ biểu đồ thể hiện tần suất xuất hiện của các items

In [53]:

import plotly.express as px
df_table["all"] = "Top 50 items" 
fig = px.treemap(df_table.head(50), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['items'],
                  color_continuous_scale='Reds',
                )
fig.show()

In [54]:

transaction = []
for i in range(dataset.shape[0]):
    transaction.append([str(dataset.values[i,j]) for j in range(dataset.shape[1])])
transaction = np.array(transaction)
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [55]:
# select top 30 items
first30 = df_table["items"].head(30).values 
# Extract Top 30
dataset = dataset.loc[:,first30] 
# size of the dataset
dataset.shape

(1203, 30)

Tìm item header table, tùy chỉnh min_support

In [56]:

from mlxtend.frequent_patterns import fpgrowth
#running the fpgrowth algorithm
res=fpgrowth(dataset,min_support=0.05, use_colnames=True)
# printing top 10
res.head(10)

,support,itemsets
0,0.196176,(eggs)
1,0.079800,(burgers)
2,0.073982,(turkey)
3,0.242727,(mineral water)
4,0.153782,(green tea)
5,0.134663,(milk)
6,0.052369,(whole wheat rice)
7,0.078969,(low fat yogurt)
8,0.163757,(french fries)
9,0.067332,(soup)


Xác định các item set phổ biến

In [57]:
# importing required module
from mlxtend.frequent_patterns import association_rules
# creating asssociation rules
res=association_rules(res, metric="lift", min_threshold=1)
# printing association rules
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(eggs),(mineral water),0.196176,0.242727,0.055694,0.283898,1.169622,0.008077,1.057494,0.180416
1,(mineral water),(eggs),0.242727,0.196176,0.055694,0.229452,1.169622,0.008077,1.043185,0.191507
2,(spaghetti),(mineral water),0.181214,0.242727,0.057357,0.316514,1.303993,0.013371,1.107957,0.284720
3,(mineral water),(spaghetti),0.242727,0.181214,0.057357,0.236301,1.303993,0.013371,1.072133,0.307848
4,(chocolate),(mineral water),0.189526,0.242727,0.060682,0.320175,1.319079,0.014679,1.113925,0.298462
5,(mineral water),(chocolate),0.242727,0.189526,0.060682,0.250000,1.319079,0.014679,1.080632,0.319429


In [58]:
# Sort values based on confidence
res.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,(chocolate),(mineral water),0.189526,0.242727,0.060682,0.320175,1.319079,0.014679,1.113925,0.298462
2,(spaghetti),(mineral water),0.181214,0.242727,0.057357,0.316514,1.303993,0.013371,1.107957,0.284720
0,(eggs),(mineral water),0.196176,0.242727,0.055694,0.283898,1.169622,0.008077,1.057494,0.180416
5,(mineral water),(chocolate),0.242727,0.189526,0.060682,0.250000,1.319079,0.014679,1.080632,0.319429
3,(mineral water),(spaghetti),0.242727,0.181214,0.057357,0.236301,1.303993,0.013371,1.072133,0.307848
1,(mineral water),(eggs),0.242727,0.196176,0.055694,0.229452,1.169622,0.008077,1.043185,0.191507
